In [455]:
"""
Created on Sat Oct 12 12:47:21 2019

@author: Noman Ashraf and Maaz Amjad

@code: Abusive Language Classification

@paper: Hairy at the Heel: Abusive Language Detection in Urdu Tweets with Analysis of Count-Based and Embedding-Based Features

"""

'\nCreated on Sat Oct 12 12:47:21 2019\n\n@author: Noman Ashraf and Maaz Amjad\n\n@code: Abusive Language Classification\n\n@paper: Hairy at the Heel: Abusive Language Detection in Urdu Tweets with Analysis of Count-Based and Embedding-Based Features\n\n'

In [516]:
import warnings
warnings.filterwarnings("ignore")

import numpy
import numpy as np
import pandas as pd
import itertools
import os
import csv
import re
import nltk
from nltk.tokenize import sent_tokenize

# Import adjustText, initialize list of texts
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA



from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm


from gensim import models
from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models.fasttext import FastText, load_facebook_vectors
from gensim.models.keyedvectors import KeyedVectors


import seaborn as sns
import matplotlib.pyplot as plt


from keras import backend
from keras import backend as K
from keras import models
from keras.models import Sequential
from keras.layers import Dense,LSTM, Dropout, Flatten, Embedding, Bidirectional, GlobalAveragePooling1D
from keras.layers import Conv1D,Conv2D, MaxPooling2D, MaxPooling1D, GlobalMaxPooling1D

from keras.optimizers import SGD,RMSprop,Adam
from keras.regularizers import l2,l1

from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.utils.vis_utils import model_to_dot


from IPython.display import SVG


#Seed Random Numbers with the TensorFlow Backend
from numpy.random import seed
seed(1)

In [597]:
train_data = "train_ds.csv"
test_data = "test_ds.csv"
w2v_file_fast_text = "cc.ur.300.bin.gz"
w2v_file_glove = "glove_model2.txt"

In [518]:
#load data
df_train = pd.read_csv(train_data, usecols=['Tweets', 'label'])
#df_train = df_train.sample(frac=1).reset_index(drop=True)
#df_train.to_csv("ds.csv", sep=',', encoding='utf-8', index=False)

df_train.head()
rows,col = df_train.shape

In [519]:
yeswisedf = df_train[(df_train['label'] == 1)]
yeswisedf.head()

,Tweets,label
0,بھگوڑی لیگ پر پورے کشمیروں کی لعنت,1
1,ہجڑے کو ہجڑا نا کہوں تو کیا کہوں دختر زرداری,1
2,گشتی کے بچے، دروازے کی چابیاں تیرے باپ باجوہ ...,1
3,کسی گشتی کے پتر ہیں یہ حرامی کافر مادرچود,1
6,ماں سے پوچھ کتی کے بچے، کس گشتے سے مروا کر آئ...,1


In [520]:
nowisedf = df_train[df_train['label'] == 0]
nowisedf.head()

,Tweets,label
4,عبدالقدیر خان نے کہا کارگل جنگ میں ہم دہلی صرف...,0
5,تو پھر ایک جرنل کے تین سال پر اتنی بکواس کیوں؟,0
9,کراچی میں پانچ سے دس لوگ بارشوں کی وجہ سے جاں ...,0
10,یہ تو اس وقت سوچنا چاہییے تھا جب عوام کو دونوں...,0
11,گرین ٹاؤن ستو کتلہ کی فردوس کوثر بی بی جس کے م...,0


In [521]:
df_row = pd.concat([yeswisedf['Tweets'], nowisedf['Tweets']])

In [522]:
df_row.head()

0                  بھگوڑی لیگ پر پورے کشمیروں کی لعنت 
1        ہجڑے کو ہجڑا نا کہوں تو کیا کہوں دختر زرداری 
2     گشتی کے بچے، دروازے کی چابیاں تیرے باپ باجوہ ...
3          کسی گشتی کے پتر ہیں یہ حرامی کافر مادرچود  
6     ماں سے پوچھ کتی کے بچے، کس گشتے سے مروا کر آئ...
Name: Tweets, dtype: object

In [523]:
print("Total Training Dataset:" + str(len(df_row)))
print("yes class:" + str(len(yeswisedf)))
print("no class:" + str(len(nowisedf)))

Total Training Dataset:2800
yes class:1400
no class:1400


In [524]:
df_test = pd.read_csv(test_data, usecols=['Tweets', 'label'])
#df_test = df_test.sample(frac=1).reset_index(drop=True)
#df_test.to_csv("ds.csv", sep=',', encoding='utf-8', index=False)

In [525]:
df_full = pd.concat([df_train, df_test])

In [526]:
abusive_df = df_full[(df_full['label'] == 1)]
abusive_df['word_count'] = abusive_df['Tweets'].apply(lambda x: len(str(x).split()))
abusive_df['char_count'] = abusive_df['Tweets'].str.len() ## this also includes spaces


df_abusive_stat = abusive_df[['word_count', 'char_count']].agg(['sum','mean'])

print("=========== Dataset Abusive ===========")
print(abusive_df.shape)
print(df_abusive_stat)
print("=========== Dataset Abusive ===========")


=========== Dataset Abusive ===========
(1750, 4)
        word_count     char_count
sum   26378.000000  118512.000000
mean     15.073143      67.721143
=========== Dataset Abusive ===========


In [527]:
non_abusive_df = df_full[(df_full['label'] == 0)]
non_abusive_df['word_count'] = non_abusive_df['Tweets'].apply(lambda x: len(str(x).split()))
non_abusive_df['char_count'] = non_abusive_df['Tweets'].str.len() ## this also includes spaces


df_non_abusive_stat = non_abusive_df[['word_count', 'char_count']].agg(['sum','mean'])

print("=========== Dataset non-Abusive ===========")
print(non_abusive_df.shape)
print(df_non_abusive_stat)
print("=========== Dataset non-Abusive ===========")

=========== Dataset non-Abusive ===========
(1750, 4)
      word_count     char_count
sum    30709.000  140627.000000
mean      17.548      80.358286
=========== Dataset non-Abusive ===========


In [528]:
keywords_dictionary = []
sentences_corpus = []
for index,row in df_full.iterrows():
    text = str(row['Tweets'])            
    sentences_corpus.append(text)
    list_of_words = text.split(" ")
    keywords_dictionary.append(list_of_words)
            
print(keywords_dictionary[0])
    


['', 'بھگوڑی', 'لیگ', 'پر', 'پورے', 'کشمیروں', 'کی', 'لعنت', '']


In [529]:
vocab = []
for kl in keywords_dictionary: 
    for w in kl:
        vocab.append(str(w))
            
corpus = []
for kl in sentences_corpus: 
    corpus = corpus + kl.split(',')

### Features

In [561]:
words = True 
#words = False 
#for char ...use words = False 
_ngram_range = (2,2)
_max_features = 100
#_max_features = None

In [562]:
if words:
    vectorizer = CountVectorizer(ngram_range=_ngram_range, max_features=_max_features)
    Count_Vect = vectorizer.fit_transform(corpus)
else:
    vectorizer = CountVectorizer(ngram_range=_ngram_range, token_pattern = r"(?u)\b\w+\b",  analyzer='char')
    Count_Vect = vectorizer.fit_transform(corpus)

In [563]:
vectorizer = TfidfVectorizer(ngram_range=_ngram_range,max_features=_max_features) # You can still specify n-grams here.
X = vectorizer.fit_transform(corpus).toarray() 

In [564]:
print( "Shape of final Ngram vector:" + str(X.shape))

Shape of final Ngram vector:(3500, 100)


#### shape represents... 3500 == total rows
#### 2nd element == # of features

In [565]:
xTrain, yTrain = X[:rows], df_train.label

In [566]:
xTest, yTest = X[rows:], df_test.label

In [567]:
print( "Shape of train vector:" + str(xTrain.shape))
print( "Shape of train labels:" + str(yTrain.shape))

Shape of train vector:(2800, 100)
Shape of train labels:(2800,)


In [568]:
print( "Shape of test vector:" + str(xTest.shape))
print( "Shape of test labels:" + str(yTest.shape))

Shape of test vector:(700, 100)
Shape of test labels:(700,)


In [569]:
def Shape_Reshaper(_xTrain, _xTest):    
    xTrain_arr = _xTrain
    xTest_arr = _xTest


    dim1,dim3 = _xTrain.shape
    dim2 = 1        

    xTrain = np.reshape(_xTrain, (dim1, dim2 , dim3))
    print(xTrain.shape)       

    t_dim1,t_dim3 = _xTest.shape
    t_dim2 = 1
    xTest = xTest_arr.reshape(t_dim1, t_dim2 , t_dim3)
    print(xTest.shape)

    _input_shape = (dim2,dim3)
    print(_input_shape)


    return xTrain,xTest,_input_shape

### Machine Learning Classifiers

In [570]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [571]:
def SVM_Ngrams(xTrain, xTest, yTrain, yTest, _C = [0.1, 1, 10], _kernel= ['linear', 'rbf'], _tol = [0.01, 0.02], _class_weight = 'balanced'):
    print("--------------- Support Vector Machine ---------------")
    print("Fitting the classifier to the training set")
    param_grid = {'C': _C, 'kernel': _kernel, 'tol': _tol}
    clf = GridSearchCV(svm.SVC(probability=True,class_weight='balanced'), param_grid)
    clf.fit(xTrain, yTrain)
    print("Best estimator found by grid search:")
    print(clf.best_estimator_)
    y_pred = clf.predict(xTest)
    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, y_pred)
    print('Accuracy: %f' % accuracy)


    print("=== Classification Report SVM ===")
    print(classification_report(yTest, y_pred))
    print('\n')
    print("=== Confusion Matrix SVM ===")
    print(confusion_matrix(yTest, y_pred))
    print('\n')
    
    print("=== ROC Score SVM ===")
    print(roc_auc_score(yTest, y_pred))
    
    
    print("--------------- Parameters ---------------")
    print_results(clf)
    
    
    print("--------------- Support Vector Machine ---------------")
    return clf

In [572]:
def RF_Ngrams(xTrain, xTest, yTrain, yTest, _n_jobs=3, _criterion = 'gini', _verbose=10):
    print("--------------- Random Forest ---------------")
    
    rfc = RandomForestClassifier( max_features=None,  n_jobs=_n_jobs, criterion = _criterion)
    parameters = {
        'n_estimators': [5,50,250],
        'max_depth': [10, 15, 20, 30]
    }

    clf = GridSearchCV(rfc, parameters)
    clf.fit(xTrain, yTrain)
    
    print(clf)
    print("Predicting labels for test data..")
    rfc_predict  = clf.predict(xTest)
    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, rfc_predict)
    print('Accuracy: %f' % accuracy)
    print("=== Classification Report Random Forest ===")
    print(classification_report(yTest, rfc_predict))
    print('\n')
    print("=== Confusion Matrix Random Forest ===")
    print(confusion_matrix(yTest, rfc_predict))
    print('\n')
    
    print("=== ROC Score RF ===")
    print(roc_auc_score(yTest, rfc_predict))
    
    print("--------------- Parameters ---------------")
    print_results(clf)
    
    print("--------------- Random Forest ---------------")
    return rfc


In [573]:
# def GB_Ngrams(xTrain, xTest, yTrain, yTest):
#     print("--------------- Gradient Boosting ---------------")
    
#     gb = GradientBoostingClassifier()
#     parameters = {
#         'n_estimators': [5, 50, 250, 500],
#         'max_depth': [1, 3, 5, 7, 9],
#         'learning_rate': [0.01, 0.1, 1, 10, 100]
#     }

#     cv = GridSearchCV(gb, parameters, cv=5)
#     cv.fit(xTrain, yTrain)
#     print("Predicting labels for test data..")
#     gbc_predict  = gb.predict(xTest)

#     print("=== Classification Report Gradient Boosting  ===")
#     print(classification_report(yTest, gbc_predict))
#     print('\n')
#     print("=== Confusion Matrix Gradient Boosting ===")
#     print(confusion_matrix(yTest, gbc_predict))
#     print('\n')
#     print("--------------- Gradient Boosting ---------------")
#     return gb

In [574]:
def Naive_Bayes_Ngrams(xTrain, xTest, yTrain, yTest):
    print("--------------- Naive Bayes ---------------")      
    model =  GaussianNB()
    model.fit(xTrain, yTrain)
    predictions = model.predict(xTest)

    
    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, predictions)
    print('Accuracy: %f' % accuracy)
    print("=== Classification Report Naive Bayes ===")
    print(classification_report(yTest, predictions))
    print('\n')
    print("=== Confusion Matrix Naive Bayes ===")
    print(confusion_matrix(yTest, predictions))
    print('\n')
    
    print("=== ROC Score Naive Bayes ===")
    print(roc_auc_score(yTest, predictions, average='weighted'))
    
   
    print("--------------- Naive Bayes ---------------")
    return model

#print("=== ROC Curve Naive Bayes ===")
# fpr, tpr, thresholds = roc_curve(yTest, predictions, pos_label=None)
#     print(fpr)
#     print(tpr)
#     print(thresholds)

In [575]:
def DT_Ngrams(xTrain, xTest, yTrain, yTest, _n_estimators = 100, _n_jobs=3, _criterion = 'gini', _max_depth  = 15, _verbose=10):
    print("--------------- Decision Tree ---------------")
    
    dt = DecisionTreeClassifier(random_state=0)
    dt = dt.fit(xTrain, yTrain)
    print("Predicting labels for test data..")
    dt_predict  = dt.predict(xTest)

    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, dt_predict)
    print('Accuracy: %f' % accuracy)
    print("=== Classification Decision Tree ===")
    print(classification_report(yTest, dt_predict))
    print('\n')
    print("=== Confusion Matrix Decision Tree ===")
    print(confusion_matrix(yTest, dt_predict))
    
    
    print("=== ROC Score DT ===")
    print(roc_auc_score(yTest, dt_predict))
    
    print('\n')
    print("--------------- Decision Tree ---------------")
    return dt

In [576]:
def AdaBoost_Ngrams(xTrain, xTest, yTrain, yTest):
    print("--------------- Adaboost ---------------")
    
    ad = AdaBoostClassifier()
    parameters = {
        'n_estimators': [5, 50, 250, 500]
    }

    clf = GridSearchCV(ad, parameters, cv=5)
    clf.fit(xTrain, yTrain)

    print("Predicting labels for test data..")
    ad_predict  = clf.predict(xTest)

    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, ad_predict)
    print('Accuracy: %f' % accuracy)
    print("=== Classification Adaboost ===")
    print(classification_report(yTest, ad_predict))
    print('\n')
    print("=== Confusion Matrix Adaboost ===")
    print(confusion_matrix(yTest, ad_predict))
    print('\n')
    
    print("=== ROC Score AD ===")
    print(roc_auc_score(yTest, ad_predict))
    
    
    print("--------------- Parameters ---------------")
    print_results(clf)
    
    print("--------------- Adaboost ---------------")
    return ad

In [577]:
def LR_Ngrams(xTrain, xTest, yTrain, yTest):
    print("--------------- Logistic Regression ---------------")
    
    lr = LogisticRegression()
    parameters = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }

    clf = GridSearchCV(lr, parameters, cv=5)
    clf.fit(xTrain, yTrain)
    
    print("Predicting labels for test data..")
    lr_predict  = clf.predict(xTest)

    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, lr_predict)
    print('Accuracy: %f' % accuracy)
    print("=== Classification Logistic Regression ===")
    print(classification_report(yTest, lr_predict))
    print('\n')
    print("=== Confusion Matrix Logistic Regression ===")
    print(confusion_matrix(yTest, lr_predict))
    print('\n')
    
    print("=== ROC Score LR ===")
    print(roc_auc_score(yTest, lr_predict))
    
    print("--------------- Parameters ---------------")
    print_results(clf)
    
    print("--------------- Logistic Regression ---------------")
    return lr

In [578]:
rf_ngram = RF_Ngrams(xTrain, xTest, yTrain, yTest)

--------------- Random Forest ---------------
GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=3,
                                              oob_score=False,
       

In [511]:
lf_ngram = LR_Ngrams(xTrain, xTest, yTrain, yTest)

--------------- Logistic Regression ---------------
Predicting labels for test data..
=== AUC Score ===
Accuracy: 0.835714
=== Classification Logistic Regression ===
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       350
           1       0.86      0.80      0.83       350

    accuracy                           0.84       700
   macro avg       0.84      0.84      0.84       700
weighted avg       0.84      0.84      0.84       700



=== Confusion Matrix Logistic Regression ===
[[305  45]
 [ 70 280]]


=== ROC Score LR ===
0.8357142857142857
--------------- Parameters ---------------
BEST PARAMS: {'C': 1}

0.809 (+/-0.054) for {'C': 0.001}
0.809 (+/-0.053) for {'C': 0.01}
0.817 (+/-0.051) for {'C': 0.1}
0.821 (+/-0.035) for {'C': 1}
0.804 (+/-0.024) for {'C': 10}
0.774 (+/-0.048) for {'C': 100}
0.753 (+/-0.033) for {'C': 1000}
--------------- Logistic Regression ---------------


In [512]:
adb_ngram =  AdaBoost_Ngrams(xTrain, xTest, yTrain, yTest)

--------------- Adaboost ---------------
Predicting labels for test data..
=== AUC Score ===
Accuracy: 0.808571
=== Classification Adaboost ===
              precision    recall  f1-score   support

           0       0.76      0.91      0.83       350
           1       0.88      0.71      0.79       350

    accuracy                           0.81       700
   macro avg       0.82      0.81      0.81       700
weighted avg       0.82      0.81      0.81       700



=== Confusion Matrix Adaboost ===
[[317  33]
 [101 249]]


=== ROC Score AD ===
0.8085714285714285
--------------- Parameters ---------------
BEST PARAMS: {'n_estimators': 50}

0.689 (+/-0.035) for {'n_estimators': 5}
0.794 (+/-0.016) for {'n_estimators': 50}
0.785 (+/-0.02) for {'n_estimators': 250}
0.768 (+/-0.032) for {'n_estimators': 500}
--------------- Adaboost ---------------


In [547]:
nb_ngram = Naive_Bayes_Ngrams(xTrain, xTest, yTrain, yTest)

--------------- Naive Bayes ---------------
=== AUC Score ===
Accuracy: 0.721429
=== Classification Report Naive Bayes ===
              precision    recall  f1-score   support

           0       0.86      0.53      0.65       350
           1       0.66      0.91      0.77       350

    accuracy                           0.72       700
   macro avg       0.76      0.72      0.71       700
weighted avg       0.76      0.72      0.71       700



=== Confusion Matrix Naive Bayes ===
[[185 165]
 [ 30 320]]


=== ROC Score Naive Bayes ===
0.7214285714285714
--------------- Naive Bayes ---------------


In [514]:
dt_ngram = DT_Ngrams(xTrain, xTest, yTrain, yTest)

--------------- Decision Tree ---------------
Predicting labels for test data..
=== AUC Score ===
Accuracy: 0.784286
=== Classification Decision Tree ===
              precision    recall  f1-score   support

           0       0.78      0.80      0.79       350
           1       0.79      0.77      0.78       350

    accuracy                           0.78       700
   macro avg       0.78      0.78      0.78       700
weighted avg       0.78      0.78      0.78       700



=== Confusion Matrix Decision Tree ===
[[280  70]
 [ 81 269]]
=== ROC Score DT ===
0.7842857142857144


--------------- Decision Tree ---------------


In [515]:
svm_ngram = SVM_Ngrams(xTrain, xTest, yTrain, yTest)

--------------- Support Vector Machine ---------------
Fitting the classifier to the training set


KeyboardInterrupt: 

In [ ]:
#gb_ngram =  GB_Ngrams(xTrain, xTest, yTrain, yTest)

### Word Embeddings

In [549]:
#%% fast load
w2vmodel = FastText.load_fasttext_format(w2v_file_fast_text)   
print("Word 2 Vector File Loaded!")   

Word 2 Vector File Loaded!


#### GLOVE Embeddings

In [598]:
from gensim.models.keyedvectors import KeyedVectors
#%% Glove load
w2vmodel = KeyedVectors.load_word2vec_format(w2v_file_glove, binary=False)
#w2vmodel = KeyedVectors.load_word2vec_format(w2v_file_glove)
print("Word 2 Vector File Loaded!")   

Word 2 Vector File Loaded!


In [557]:
def LSTM_Ngrams(_xTrain, _xTest, yTrain, yTest, _loss='mean_squared_error', _optimizer= 'Adam', _metrics=['accuracy'], _epochs = 25 , _validation_split = 0.2, _batch_size = 2, _verbose = 0 ):
    print("--------------- LSTM ---------------")       

    xTrain,xTest,_input_shape = Shape_Reshaper(_xTrain, _xTest)

    model = Sequential()
    model.add(LSTM(4, input_shape=_input_shape, activation='tanh' , return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten()) 
    model.add(Dense(1 ,activation='sigmoid'))
    model.summary()
    model.compile(loss=_loss, optimizer= _optimizer, metrics=_metrics)
    history = model.fit(xTrain, yTrain, epochs=_epochs,  validation_split = _validation_split)

    accuracy = model.evaluate(xTest, yTest,  verbose= _verbose)
    y_pred = model.predict_classes(xTest, batch_size = _batch_size, verbose = _verbose)
    
    print("=== AUC Score ===")
    accuracy = accuracy_score(yTest, y_pred)
    print('Accuracy: %f' % accuracy)

    print("=== Classification Report LSTM ===")
    print(classification_report(yTest, y_pred))
    print('\n')
    print("=== Confusion Matrix LSTM ===")
    print(confusion_matrix(yTest, y_pred))
    print('\n')
        
    print("=== ROC Score Naive Bayes ===")
    print(roc_auc_score(yTest, y_pred, average='weighted'))
    
    print("--------------- LSTM ---------------")
    return model

In [558]:
def Generate_Vectors(data):
    vector = w2vmodel.wv['easy']
    print( "Shape of Vector:" + str(vector.shape))

 
    X_train_Vector = []
    for d in data:
        words = sent_tokenize(d) 
        vector_list = []
        for word in words:
            if word in w2vmodel.wv.vocab:
                vector_list.append(w2vmodel[word])
            else:
                vector_list.append(np.random.uniform(-0.1, 0.1, 300))

        matrix_2d = np.array(vector_list)
        average_sentence_vector = np.mean(matrix_2d, axis = 0)
        X_train_Vector.append(average_sentence_vector)

    X = numpy.array(X_train_Vector)
    print( "Shape of documents" + str(X.shape)) 
    return X

In [559]:
def CNN1D_Ngrams(_xTrain, _xTest, yTrain, yTest, _loss='mean_squared_error', _optimizer= 'Adam', _metrics=['accuracy'], _epochs = 25 , _validation_split = 0.2, _batch_size = 4, _verbose = 2 ):
    print("--------------- CNN1D ---------------")  
    
    xTrain,xTest,_input_shape = Shape_Reshaper(_xTrain, _xTest)

    model = Sequential()
    model.add(Conv1D(16, (3),strides=3, padding='same',activation='tanh', input_shape=_input_shape))
    model.add(Dense(32 ,activation='tanh'))
    model.add(Dropout(0.1))
    #model.add(Dense(64 ,activation='tanh'))
    #model.add(Dropout(0.2))
    model.add(Flatten()) 
    model.add(Dense(1 ,activation='sigmoid'))
    model.summary()
    model.compile(loss=_loss, optimizer=_optimizer, metrics=_metrics)

    model.fit(xTrain, yTrain, epochs=_epochs,  validation_split=_validation_split)

    accuracy = model.evaluate(xTest, yTest,  verbose=_verbose)
    print(accuracy)
    y_pred = model.predict_classes(xTest)

    print("=== Classification Report Conv 1D ===")
    print(classification_report(yTest, y_pred))
    print('\n')

    print("=== Confusion Matrix Conv 1D ===")
    print(confusion_matrix(yTest, y_pred))
    print('\n')

    print("--------------- CNN1D ---------------") 
    return model


In [553]:
xTrain = Generate_Vectors(df_train.Tweets)
yTrain = df_train.label

Shape of Vector:(300,)
Shape of documents(2800, 300)


In [554]:
xTest = Generate_Vectors(df_test.Tweets)
yTest = df_test.label

Shape of Vector:(300,)
Shape of documents(700, 300)


In [555]:
cnn1d_ngram = CNN1D_Ngrams(xTrain, xTest, yTrain, yTest,_epochs = 50, _verbose=2)

--------------- CNN1D ---------------
(2800, 1, 300)
(700, 1, 300)
(1, 300)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 32)             28832     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 32)             1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 64)             2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
____________________________________________

In [560]:
lstm_ngram = LSTM_Ngrams(xTrain, xTest, yTrain, yTest,_epochs = 50, _verbose=5)

--------------- LSTM ---------------
(2800, 1, 300)
(700, 1, 300)
(1, 300)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 4)              4880      
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 4)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 4,885
Trainable params: 4,885
Non-trainable params: 0
_________________________________________________________________
Train on 2240 samples, validate on 560 samples
Epoch 1/50
2240/2240 [==============================] - 1s 438us/step - loss: 0.2501 - accuracy: 0.4978 - val_loss: 0.2501 - val

Epoch 50/50
2240/2240 [==============================] - 0s 146us/step - loss: 0.2020 - accuracy: 0.6844 - val_loss: 0.2847 - val_accuracy: 0.4964
=== AUC Score ===
Accuracy: 0.531429
=== Classification Report LSTM ===
              precision    recall  f1-score   support

           0       0.53      0.54      0.53       350
           1       0.53      0.53      0.53       350

    accuracy                           0.53       700
   macro avg       0.53      0.53      0.53       700
weighted avg       0.53      0.53      0.53       700



=== Confusion Matrix LSTM ===
[[188 162]
 [166 184]]


=== ROC Score Naive Bayes ===
0.5314285714285714
--------------- LSTM ---------------
